# Abstract

<font color="blue"> What does this notebook include? </font>
1. Choose the propreate LLM:   
We need a "off-the-shelf LLM", which are going to be pre-trained models that are trained on huge amount of dataset and learned the common rules of languages.  
For here I tried: T5, BERT (GPT is also one, but not publicly available).
2. There are three mothods that we want to explore to enhance the performance of models:  
Supervised Fine Tuning (SFT);   
Reinforcement Learning with Human Feedback (RLHF);  
Reinforcement Learning with AI Feedback (RLAIF).  

<font color="red"> Why this notebook is incomplete? </font>  
1. TL;DR dataset that the paper mentioned is too large for my computer to run;
2. RLHF need lots of human efforts to giving feedbacks which is not what I am capable of doing in the short time.

# 1. Choose LLM

This model is called "off-the-shelf LLM" which is pre-trained on a huge datesets. It learned general rules of language.

In [1]:
# Install required packages

!pip install accelerate
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00

In [2]:
# Example of google/flan-t5-base
# Source: https://huggingface.co/google/flan-t5-base?text=Translate+to+German%3A++My+name+is+Arthur

# pip install accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto")

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Wie old sind Sie?</s>


In [3]:
# Example of t5-small
# Source: https://huggingface.co/t5-small
#         https://huggingface.co/docs/transformers/model_doc/t5

from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

input_ids = tokenizer("translate English to German: How old are you?", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Wie alt sind Sie?


In [4]:
# Example of bert-base-cased
# Source: https://huggingface.co/bert-base-cased

from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained("bert-base-cased")
text = "translate English to German: How old are you?"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

closer Champion σ ě [unused58] [unused1]


In [5]:
# Example of facebook/bart-base & facebook/bart-large-cnn
# Source: https://github.com/facebookresearch/fairseq/tree/main/examples/bart


# from transformers import BartForConditionalGeneration, BartTokenizer
# model_name = "facebook/bart-base"  # Pre-trained BART model for CNN/Daily Mail dataset
# model = BartForConditionalGeneration.from_pretrained(model_name)
# tokenizer = BartTokenizer.from_pretrained(model_name)

from transformers import pipeline

summarizer_base = pipeline("summarization", model="facebook/bart-base")
summarizer_large = pipeline("summarization", model="facebook/bart-large-cnn")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
print(summarizer_base(ARTICLE, max_length=130, min_length=30, do_sample=False))
print(summarizer_large(ARTICLE, max_length=130, min_length=30, do_sample=False))


#>>> [{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


[{'summary_text': ' New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York. She declared "I do" five times, sometimes only within two weeks of each other, according to court documents.She then filed for a marriage license in the Bronx, but without divorcing her first husband. She filed for permanent residence status in 2009.Only 18 days after that marriage, she became pregnant again. She got hitched yet again. Then, Barrientios declared "you do" four more times, and again, she filed for residency status in 2010.In 2010, she married'}]
[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


In [6]:
summarizer_t5 = pipeline("summarization", model="t5-large")

print(summarizer_t5(ARTICLE, max_length=130, min_length=30, do_sample=False))

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'Liana barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree" in an application for a marriage license, she stated it was her "first and only" marriage .'}]


In [7]:
## We will use t5-large & facebook/bart-large-cnn as our example models ("off-the-shelf LLM")

# 2. Three methods to enhance the performance of models

## 2.1 SFT

In [8]:
from transformers import Trainer, TrainingArguments

In [9]:
from datasets import load_dataset
from trl import SFTTrainer

dataset = load_dataset("UCL-DARK/openai-tldr-filtered")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary'],
        num_rows: 116722
    })
    validation: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary'],
        num_rows: 6447
    })
    test: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary'],
        num_rows: 6553
    })
})

### Run the whole dataset

The Problem is, I do not have great GPU so it runs slow.

In [ ]:
trainer_t5 = SFTTrainer(
    "facebook/bart-large-cnn",
    train_dataset=dataset["train"],
    dataset_text_field="post",
    max_seq_length=512,
    packing=True)
trainer_t5.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:125: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.520800
1000,0.017200
1500,0.013500


### [Alternative Approach]

This method allow model to run a little bit (as I observed, 1 row of data) at a time. It will let users be able to peek the result a little bit without running whole data.

In [7]:
iterable_dataset_train = dataset["train"].to_iterable_dataset()
iterable_dataset_validation = dataset["validation"].to_iterable_dataset()
iterable_dataset_test = dataset["test"].to_iterable_dataset()

In [21]:
training_args = TrainingArguments(
    max_steps=116722,
    output_dir="."
)
# model = my model...

trainer = SFTTrainer(
    model="facebook/bart-large-cnn",
    args=training_args,
    train_dataset=iterable_dataset_train,
    dataset_text_field="post",
    max_seq_length=512
                    # rest of the arguments...
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:125: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(


In [17]:
print(next(iter(iterable_dataset_train)))

{'id': 't3_1hxu8s', 'subreddit': 'relationships', 'title': 'I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting', 'post': "Not sure if this belongs here but it's worth a try. \n\nBackstory:\nWhen I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand  it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going to a festival with my friends. When I think back I wish he just ended. So after he ended it added my depression I suffered but my friends helped me through it and I got rid of everything from him along with cutting contact. \n\nNow: Its been almost 3 years now and I've gotten better after counselling and mild anti depr

## 2.2 RLHF

## 2.3 RLAIF